In [2]:
import numpy as np
import pandas as pd
import scipy.linalg as la
import scipy.optimize as opt
import copy
np.set_printoptions(threshold=np.inf, suppress=True, formatter={'float': '{: 0.8f}'.format}, linewidth=75)

In [3]:
def kron(A, B):
    return la.kron(A, B)

In [4]:
def kronsum(A, B):
    if A.shape[0] != A.shape[1]:
        raise ValueError('A is not square')

    if B.shape[0] != B.shape[1]:
        raise ValueError('B is not square')
    
    L = kron(A, np.eye(B.shape[0]))
    R = kron(np.eye(A.shape[0]), B)
    
    return L+R

In [5]:
n = 3                   # Количество матриц D_k
p_num = 100             # Количество векторов стац. вероятн. p
eps_G = 10 ** (-6)      # Точность нахождения матр. G
eps_Phi = 10 ** (-6)    # Точность нахождения матр. Phi
eps_p_i = 10 ** (-6)    # Точность нахождения векторов p

In [6]:
# Входной BMAP
matrD_0 = np.array([[-86., 0.01], [0.02, -2.76]]) # / 35
matrD = np.array([[85, 0.99], [0.2, 2.54]]) # / 35
matrD_k = [matrD_0]

W_ = matrD_0.shape[0]
W = W_ - 1

q = 0.8
for k in range(1, n+1):
    matrD_k.append(matrD * (q ** (k-1)) * (1 - q) / (1 - q ** 3))

for matr in matrD_k:
    print(matr)

[[-86.00000000  0.01000000]
 [ 0.02000000 -2.76000000]]
[[ 34.83606557  0.40573770]
 [ 0.08196721  1.04098361]]
[[ 27.86885246  0.32459016]
 [ 0.06557377  0.83278689]]
[[ 22.29508197  0.25967213]
 [ 0.05245902  0.66622951]]


In [7]:
# Характеристики входного BMAP

matrD_1_ = np.zeros(matrD_k[0].shape)
for matr in matrD_k:
    matrD_1_ += matr
print('matrD_1_ =', matrD_1_)
matr_a = np.array(copy.deepcopy(matrD_1_))
for i in range(matr_a.shape[0]):
    matr_a[i][0] = 1

matr_b = np.zeros((matr_a.shape[0], 1))
matr_b[0][0] = 1

matr_a = np.transpose(matr_a)

theta = la.solve(matr_a, matr_b).reshape(-1)    # Алгоритм проверен.

matrdD_1_ = np.array(copy.deepcopy(matrD_k[1]))

print('Theta =', theta)

for i in range(2, n+1):
    matrdD_1_ += matrD_k[i] * i
vect_e = np.array([[1.] for i in range(0, matrD_1_.shape[0])])
lamD = np.dot(np.dot(theta, matrdD_1_), vect_e)
print('lambda =', lamD[0])

matrD_1_ = [[-1.00000000  1.00000000]
 [ 0.22000000 -0.22000000]]
Theta = [ 0.18032787  0.81967213]
lambda = 32.8853990863


In [8]:
# Поток поломок MAP
matrH0 = np.array([[-8.110725, 0], [0, -0.26325]])
matrH1 = np.array([[8.0568, 0.053925], [0.146625, 0.116625]])
V_ = matrH1.shape[0]
V = V_ - 1
matrH = matrH0 + matrH1
matr_a = copy.deepcopy(matrH)
for i in range(matr_a.shape[0]):
    # print(matr_a)
    matr_a[i][0] = 1
    
matr_b = np.zeros((matr_a.shape[0], 1))
matr_b[0][0] = 1

matr_a = np.transpose(matr_a)

print(matr_a)

gamma = la.solve(matr_a, matr_b).reshape(-1)

vect_e = np.array([[1.] for i in range(0, matrD_1_.shape[0])])
h = np.dot(np.dot(gamma, matrH1), vect_e)
print('h =', h[0])

[[ 1.00000000  1.00000000]
 [ 0.05392500 -0.14662500]]


h = 6.00065225318


In [9]:
# Поток обслуживания PH1
beta1 = np.array([[1, 0]])
matrS1 = np.array([[-20, 20], [0, -20]])
M1 = matrS1.shape[0]
M1_ = M1 + 1
M1_e = np.array([[1], [1]])
matrS1_0 = np.dot(- matrS1, M1_e)
vect_e = np.array([[1.] for i in range(0, matrS1.shape[0])])
# print(np.dot(beta1, la.inv(matrS1)))
mu_1 = -la.inv(np.dot(np.dot(beta1, la.inv(matrS1)), vect_e))
print(mu_1)

[[ 10.00000000]]


In [10]:
# Поток обслуживания PH2
beta2 = np.array([[1, 0]])
matrS2 = np.array([[-2, 2], [0, -2]])
M2 = matrS2.shape[0]
M2_ = M2 + 1
M2_e = np.array([[1], [1]])
matrS2_0 = np.dot(- matrS2, M2_e)

vect_e = np.array([[1.] for i in range(0, matrS2.shape[0])])
mu_2 = -la.inv(np.dot(np.dot(beta2, la.inv(matrS2)), vect_e))
print(mu_2)

[[ 1.00000000]]


In [11]:
matrS_w = kron(np.dot(matrS1_0, beta1), 
               np.dot(M2_e, beta2)) + kron(np.dot(M1_e, beta1), np.dot(matrS2_0, beta2))

In [12]:
# Поток переключения с прибора-1 на прибор-2
alpha1 = np.array([[0.05, 0.95]])
matrA1 = np.array([[-1.86075, 0.], [0., -146.9994]])
L1 = matrA1.shape[0]
L1_ = L1 + 1
L1_e = np.array([[1], [1]])
matrA1_0 = - np.dot(matrA1, L1_e)

vect_e = np.array([[1.] for i in range(0, matrA1.shape[0])])
kappa_1 = -la.inv(np.dot(np.dot(alpha1, la.inv(matrA1)), vect_e))
print(kappa_1)

[[ 29.99985287]]


In [13]:
# Поток переключения с прибора-2 на прибор-1
alpha2 = np.array([[0.05, 0.95]])
matrA2 = np.array([[-5.58225, 0.], [0., -440.9982]])
L2 = matrA2.shape[0]
L2_ = L2 + 1
L2_e = np.array([[1], [1]])
matrA2_0 = - np.dot(matrA2, L2_e)

vect_e = np.array([[1.] for i in range(0, matrA1.shape[0])])
kappa_2 = -la.inv(np.dot(np.dot(alpha2, la.inv(matrA2)), vect_e))
print(kappa_2)

[[ 89.99955862]]


In [14]:
# Поток ремонта PH
tau = np.array([[0.98, 0.02]])
matrT = np.array([[-100., 0.], [0., -0.0002]])
T_e = np.array([[1], [1]])
matrT0 = - np.dot(matrT, T_e)

R = matrT.shape[0]
R_ = R + 1
vect_e = np.array([[1.] for i in range(0, matrT.shape[0])])
phi = -np.dot(np.dot(tau, la.inv(matrT)), vect_e)
print(phi)

[[ 100.00980000]]


In [15]:
a = W_ * V_
print('a =', a)

a = 4


In [16]:
# Q~0
block00 = kronsum(matrD_k[0], matrH0)
block03 = kron(kron(kron(np.eye(W_), matrH1), tau), alpha1)
block10 = kron(np.eye(a), matrA2_0)
block11 = kronsum(kronsum(matrD_k[0], matrH0), matrA2)
block12 = kron(kron(kron(np.eye(W_), matrH1), tau), L2_e)
block21 = kron(kron(np.eye(a), matrT0), alpha2)
block22 = kronsum(kronsum(matrD_k[0], matrH), matrT)
block30 = kron(kron(np.eye(a), matrT0), L1_e)
block32 = kron(kron(np.eye(a), np.eye(R)), matrA1_0)
block33 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrT), matrA1)
block01 = np.zeros((block00.shape[0], block11.shape[1]))
block02 = np.zeros((block00.shape[0], block12.shape[1]))
block13 = np.zeros((block10.shape[0], block03.shape[1]))
block20 = np.zeros((block21.shape[0], block10.shape[1]))
block23 = np.zeros((block21.shape[0], block03.shape[1]))
block31 = np.zeros((block30.shape[0], block11.shape[1]))

print(block30.shape)

matrQw_0 = np.bmat([[block00, block01, block02, block03],
                    [block10, block11, block12, block13], 
                    [block20, block21, block22, block23],
                    [block30, block31, block32, block33]])
print(matrQw_0.shape)

(16, 4)
(36, 36)


In [17]:
# Q~k
matrQw_k = [matrQw_0]
for i in range(1, n+1):
    block0 = kron(kron(matrD_k[i], np.eye(V_)), beta1)
    block1 = kron(kron(kron(matrD_k[i], np.eye(V_)), beta2), np.eye(L2))
    block2 = kron(kron(kron(matrD_k[i], np.eye(V_)), beta2), np.eye(R))
    block3 = kron(kron(matrD_k[i], np.eye(V_)), np.eye(R*L1))
    matr_temp = la.block_diag(block0, block1, block2, block3)
    matrQw_k.append(matr_temp)
    
print(matrQw_k[0].shape)
print(matrQw_k[1].shape)

for i in range(matrQw_k[0].shape[0]):
    sum = 0
    for matr in matrQw_k:
        sum += np.sum(matr[i])
    # print('matrQw_k[' + str(i) + '] = ', sum)

(36, 36)
(36, 56)


In [18]:
# Q^0
block0 = kron(np.eye(a), matrS1_0)
block1 = kron(kron(np.eye(a), matrS2_0), np.eye(L2))
block2 = kron(kron(np.eye(a), matrS2_0), np.eye(R))
block3 = np.zeros((a*R*L1, a*R*L1))
matrQv_0 = la.block_diag(block0, block1, block2, block3)

print(matrQv_0.shape)

(56, 36)


In [19]:
# Q_0
block0 = kron(np.eye(a), np.dot(matrS1_0, beta1))
block1 = kron(kron(np.eye(a), np.dot(matrS2_0, beta2)), np.eye(L2))
block2 = kron(kron(np.eye(a), np.dot(matrS2_0, beta2)), np.eye(R))
block3 = np.zeros((a*R*L1, a*R*L1))
matrQ_0 = la.block_diag(block0, block1, block2, block3)

print(matrQ_0.shape)

(56, 56)


In [20]:
# Q_1
block00 = kronsum(kronsum(matrD_k[0], matrH0), matrS1)
block03 = kron(kron(kron(kron(np.eye(W_), matrH1), M1_e), tau), alpha1)
block10 = kron(kron(kron(np.eye(a), M2_e), beta1), matrA2_0)
block11 = kronsum(kronsum(kronsum(matrD_k[0], matrH0), matrS2), matrA2)
block12 = kron(kron(kron(kron(np.eye(W_), matrH1), np.eye(M2)), tau), L2_e)
block21 = kron(kron(kron(np.eye(a), np.eye(M2)), matrT0), alpha1)
block22 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrS2), matrT)
block30 = kron(kron(kron(np.eye(a), beta1), matrT0), L1_e)
block32 = kron(kron(kron(np.eye(a), beta2), np.eye(R)), matrA1_0)
block33 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrT), matrA1)
block01 = np.zeros((block00.shape[0], block11.shape[1]))
block02 = np.zeros((block00.shape[0], block12.shape[1]))
block13 = np.zeros((block10.shape[0], block03.shape[1]))
block20 = np.zeros((block21.shape[0], block10.shape[1]))
block23 = np.zeros((block21.shape[0], block03.shape[1]))
block31 = np.zeros((block30.shape[0], block11.shape[1]))

matrQ_1 = np.bmat([[block00, block01, block02, block03], 
                   [block10, block11, block12, block13], 
                   [block20, block21, block22, block23], 
                   [block30, block31, block32, block33]])

print(matrQ_1.shape)

(56, 56)


In [21]:
matrQ_k = [matrQ_0, matrQ_1]
for k in range(2, n+2):
    block0 = kron(matrD_k[k-1], np.eye(V_ * M1))
    block1 = kron(matrD_k[k-1], np.eye(V_*M2*L2))
    block2 = kron(matrD_k[k-1], np.eye(V_*M2*R))
    block3 = kron(matrD_k[k-1], np.eye(V_*R*L1))
    matr_temp = la.block_diag(block0, block1, block2, block3)
    matrQ_k.append(matr_temp)

# for i in range(matrQ_k[0].shape[0]):
#     sum = np.sum(matrQv_0[i])
#     for j in range(1, np.array(matrQ_k).shape[0]):
#         sum += np.sum(matrQ_k[j][i])
#     print('matrQ_k[' + str(i) + '] = ', sum)

In [22]:
# Проверка генератора Q

In [23]:
# Поиск матрицы G

matrQ_1_neg_inv = np.linalg.inv(-matrQ_k[1])


def calc_G(matrG_prev):
    temp_sum = np.array(copy.deepcopy(matrQ_k[0])) 
    for k in range(2, n + 2):
        temp_sum += np.dot(matrQ_k[k], np.linalg.matrix_power(matrG_prev, k))
    matrG_new = np.dot(matrQ_1_neg_inv, temp_sum)
    return matrG_new

matrG_old = np.eye(matrQ_k[1].shape[0])
matrG = calc_G(matrG_old)

i = 1
while la.norm(matrG - matrG_old, ord=np.inf) >= eps_G:
    matrG_old = matrG
    matrG = calc_G(matrG_old)
    i += 1
print(i)
print(la.norm(matrG, ord=np.inf))
print(matrG.shape)

137
1.0000011004408416
(56, 56)


In [24]:
la.norm(matrG, ord=np.inf)

1.0000011004408416

In [25]:
temp_sum = np.array(copy.deepcopy(matrQ_k[1]))
for k in range(2, n+2):
    temp_sum += np.dot(matrQ_k[k], np.linalg.matrix_power(matrG, k-1))
matrG_0 = la.inv(temp_sum)
matrG_0 = -np.dot(matrG_0, matrQv_0)
# pd.DataFrame(matrG_0)
la.norm(matrG_0, ord=np.inf)

1.0000012050895348

In [26]:
matrQ_il = []
for i in range(0, p_num):
    matrQ_il.append([])
    if i == 0:
        for l in range(0, n + 1):
            # здесь до n, т.к. нет больше матриц Q_k
            temp_matr = np.array(copy.deepcopy(matrQw_k[l]))
            for k in range(l + 1, n + 1):
                mult_matr = np.array(copy.deepcopy(matrQw_k[k]))
                for kk in range(k - 1, l - 1, -1):
                    if kk == 0:
                        mult_matr = np.dot(mult_matr, matrG_0)
                    else:
                        mult_matr = np.dot(mult_matr, matrG)
                print("mult matr 0 shape : ", mult_matr.shape)
                temp_matr += mult_matr
            matrQ_il[i].append(temp_matr)
        for l in range(n + 1, p_num):
            matrQ_il[i].append(np.zeros(matrQw_k[1].shape))
    else:
        for l in range(0, p_num):
            if l >= i and (l - i) <= (n + 1):
                if (l - i + 1) <= (n + 1): 
                    temp_matr = np.array(copy.deepcopy(matrQ_k[l - i + 1]))
                else:
                    temp_matr = np.zeros(matrQ_k[0].shape)
                    
                for k in range(l + 1, p_num):  # sum from l+1 to inf
                    if (k - i + 1) <= (n + 1):
                        mult_matr = np.array(copy.deepcopy(matrQ_k[k - i + 1]))
                        for kk in range(l, k):
                            mult_matr = np.dot(mult_matr, matrG)
                        
                        temp_matr += mult_matr
                matrQ_il[i].append(temp_matr)
            else:
                matrQ_il[i].append(np.zeros(matrQ_k[0].shape))

# print(len(matrQ_il[3]))
print(la.norm(matrQ_il[1][3], ord=np.inf))

mult matr 0 shape :  (36, 36)
mult matr 0 shape :  (36, 36)
mult matr 0 shape :  (36, 36)
mult matr 0 shape :  (36, 56)
mult matr 0 shape :  (36, 56)
mult matr 0 shape :  (36, 56)


50.74822150072864


In [27]:
matrPhi_0 = np.eye(matrQ_il[0][0].shape[0])
matrPhi_l = [matrPhi_0]
for l in range(1, p_num):
    temp_matr = np.dot(np.dot(matrPhi_l[0], matrQ_il[0][l]), la.inv(-matrQ_il[l][l]))
    for i in range(1, l):
        # print(matrPhi_l[i].dot(matrQ_il[i][l]).dot(la.inv(-matrQ_il[l][l])).shape)
        temp_matr += np.dot(np.dot(matrPhi_l[i], matrQ_il[i][l]), la.inv(-matrQ_il[l][l]))
    matrPhi_l.append(temp_matr)
print(matrPhi_l[1].shape)

(36, 56)


In [28]:
print(la.norm(matrPhi_l[p_num - 1], ord=np.inf))

1047.1679185160397


In [29]:
# Вычисление p_0
matr_a = np.array(- copy.deepcopy(matrQ_il[0][0]))
vect_eaR = np.array([[1.] for _ in range(matrPhi_l[0].shape[1])])
for i in range(1, p_num):
    vect_e = np.array([[1.] for _ in range(matrPhi_l[i].shape[1])])
    vect_eaR += np.dot(matrPhi_l[i], vect_e)

for i in range(matr_a.shape[0]):
    matr_a[i][0] = vect_eaR[i][0]

matr_b = np.zeros((matr_a.shape[0], 1))
matr_b[0][0] = 1.
matr_a = np.transpose(matr_a)
p0 = np.transpose(la.solve(matr_a, matr_b))

# print(p0)
print(p0)

[[ 0.00000016  0.00000035  0.00057630  0.00124123  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000820  0.00000564  0.00000057  0.00000021
   0.00000001  0.00000001  0.00000000  0.00000000  0.00002682  0.00003995
   0.00000096  0.00000237  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000226  0.00001798
   0.00000103  0.00000061  0.00000008  0.00000066  0.00000005  0.00000002]]


In [58]:
vect_p_l = [p0]
p_sum = np.sum(p0)
print('sum0 = ', np.sum(p0))
print('p0 = ', vect_p_l[0])
for l in range(1, p_num):
    vect_p_l.append(np.dot(vect_p_l[0], matrPhi_l[l]))
    p_sum += (np.sum(vect_p_l[l]))
    print('p' + str(l) + ' = ', vect_p_l[l])
    print('sum' + str(l) + ' = ', np.sum(vect_p_l[l]))
print('sum = ', p_sum)

sum0 =  0.00192549500286
p0 =  [[ 0.00000016  0.00000035  0.00057630  0.00124123  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000820  0.00000564  0.00000057  0.00000021
   0.00000001  0.00000001  0.00000000  0.00000000  0.00002682  0.00003995
   0.00000096  0.00000237  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000226  0.00001798
   0.00000103  0.00000061  0.00000008  0.00000066  0.00000005  0.00000002]]
p1 =  [[ 0.00000058  0.00000012  0.00000122  0.00000026  0.00013182  0.00008540
   0.00020629  0.00017920  0.00000001  0.00000001  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000329  0.00000216
   0.00000042  0.00000006  0.00000022  0.00000007  0.00000004  0.00000000
   0.00000003  0.00000005  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00001025  0.00002154  0.00000023  0.00000898
   0.00000032  0.00000138  0.00000001  0.00000066  0.00000000  0.00000002

sum95 =  0.0466495781039
p96 =  [[ 0.00019679  0.00013783  0.00029384  0.00028378  0.00863243  0.00604585
   0.01288846  0.01244718  0.00000495  0.00000330  0.00000073  0.00000009
   0.00000031  0.00000010  0.00000008  0.00000000  0.00021713  0.00014463
   0.00003193  0.00000405  0.00001353  0.00000448  0.00000353  0.00000011
   0.00001567  0.00007234  0.00000037  0.00005837  0.00000048  0.00000855
   0.00000001  0.00000780  0.00068735  0.00317546  0.00001619  0.00256284
   0.00002094  0.00037552  0.00000044  0.00034299  0.00000133  0.00001046
   0.00000118  0.00000036  0.00000004  0.00000032  0.00000006  0.00000001
   0.00005839  0.00045883  0.00005184  0.00001571  0.00000182  0.00001419
   0.00000267  0.00000049]]
sum96 =  0.0493181264837
p97 =  [[ 0.00020752  0.00014546  0.00030965  0.00029945  0.00773549  0.00549068
   0.01134652  0.01117928  0.00000523  0.00000348  0.00000077  0.00000010
   0.00000033  0.00000011  0.00000008  0.00000000  0.00020083  0.00013117
   0.00003001  0.000

In [31]:
# Условие эргодичности
block00 = kronsum(matrH0, matrS1) + kron(np.eye(V_), np.dot(matrS1_0, beta1))
block03 = kron(kron(kron(matrH1, M1_e), tau), alpha1)
block10 = kron(kron(np.eye(V_), np.dot(M2_e, beta1)), matrA2_0)
block11 = kronsum(kronsum(matrH0, matrS2), matrA2) + kron(kron(np.eye(V_), np.dot(matrS2_0, beta2)), np.eye(L2))
block12 = kron(kron(kron(matrH1, np.eye(M2)), tau), L2_e)
block21 = kron(kron(kron(np.eye(V_), np.eye(M2)), matrT0), alpha1)
block22 = kronsum(kronsum(matrH, matrS2), matrT) + kron(kron(np.eye(V_), np.dot(matrS2_0, beta2)), np.eye(R))
block30 = kron(kron(kron(np.eye(V_), beta1), matrT0), L1_e)
block32 = kron(kron(kron(np.eye(V_), beta2), np.eye(R)), matrA1_0)
block33 = kronsum(kronsum(matrH, matrT), matrA1)
block01 = np.zeros((block00.shape[0], block11.shape[1]))
block02 = np.zeros((block00.shape[0], block12.shape[1]))
block13 = np.zeros((block10.shape[0], block03.shape[1]))
block20 = np.zeros((block21.shape[0], block00.shape[1]))
block23 = np.zeros((block21.shape[0], block03.shape[1]))
block31 = np.zeros((block30.shape[0], block11.shape[1]))
matrGamma = np.bmat([[block00, block01, block02, block03],
                     [block10, block11, block12, block13],
                     [block20, block21, block22, block23],
                     [block30, block31, block32, block33]])
# pd.DataFrame(matrGamma)

In [32]:
matr_a = copy.deepcopy(np.array(matrGamma))
for i in range(matr_a.shape[0]):
    matr_a[i][0] = 1
matr_b = np.zeros((matr_a.shape[0], 1))
matr_b[0][0] = 1
matr_a = np.transpose(matr_a)

x = la.solve(matr_a, matr_b).reshape(-1)

print('x = ', x)
x1 = x[0:V_*M1]
x2 = x[V_*M1:V_*M1 + V_*M2*L2]
x3 = x[V_*M1 + V_*M2*L2: V_*M1 + V_*M2*L2 + V_*M2*R]

print('x1 = ', x1)
print('x2 = ', x2)
print('x3 = ', x3)

x =  [ 0.00068409  0.00048671  0.00041198  0.00040662  0.00001789  0.00001158
  0.00000294  0.00000048  0.00000086  0.00000025  0.00000040  0.00000006
  0.00005425  0.36492823  0.00000133  0.36488009  0.00000092  0.13403250
  0.00000002  0.13403103  0.00000459  0.00003600  0.00000501  0.00000123
  0.00000008  0.00000061  0.00000021  0.00000002]
x1 =  [ 0.00068409  0.00048671  0.00041198  0.00040662]
x2 =  [ 0.00001789  0.00001158  0.00000294  0.00000048  0.00000086  0.00000025
  0.00000040  0.00000006]
x3 =  [ 0.00005425  0.36492823  0.00000133  0.36488009  0.00000092  0.13403250
  0.00000002  0.13403103]


In [33]:
e_V_ = np.array([[1.] for i in range(0, V_)])
e_R = np.array([[1.] for i in range(0, R)])
pi1 = x1.dot(kron(e_V_, np.eye(M1)))
pi2 = x2.dot(kron(kron(e_V_, np.eye(M2)), L2_e))
pi3 = x3.dot(kron(kron(e_V_, np.eye(M2)), e_R))
print('pi1 = ', pi1)
print('pi2 = ', pi2)
print('pi3 = ', pi3)

pi1 =  [ 0.00109607  0.00089334]
pi2 =  [ 0.00003058  0.00000389]
pi3 =  [ 0.49901590  0.49891247]


In [34]:
rho = np.dot(pi1, matrS1_0) + np.dot((pi2 + pi3), matrS2_0)
print('rho = ', rho[0])

rho =  1.01569947522


In [35]:
vect_P_1_ = copy.deepcopy(vect_p_l[1])
for l in range(2, p_num):
    vect_P_1_ += vect_p_l[l]
print('P(1) =', vect_P_1_)

P(1) = [[ 0.00423720  0.00298281  0.00631097  0.00614228  0.17331829  0.12200516
   0.25709811  0.25013180  0.00010735  0.00007139  0.00001578  0.00000200
   0.00000669  0.00000221  0.00000174  0.00000006  0.00441458  0.00291732
   0.00065342  0.00008190  0.00027599  0.00009011  0.00007282  0.00000230
   0.00033917  0.00156182  0.00000800  0.00125905  0.00001033  0.00018357
   0.00000022  0.00016760  0.01386007  0.06657851  0.00032765  0.05467589
   0.00042064  0.00790052  0.00000885  0.00733513  0.00002879  0.00022599
   0.00002564  0.00000774  0.00000090  0.00000699  0.00000132  0.00000024
   0.00117641  0.00923598  0.00108187  0.00031642  0.00003659  0.00028452
   0.00005603  0.00000979]]


In [36]:
vect_dP_1_ = copy.deepcopy(vect_p_l[1])
for l in range(2, p_num):
    vect_dP_1_ += l * vect_p_l[l]

In [37]:
vect_ddP_1_ = copy.deepcopy(vect_p_l[2]) * 2
for l in range(3, p_num):
    vect_ddP_1_ += l * (l - 1) * vect_p_l[l]

In [38]:
# Среднее число запросов в системе

vect_e = np.array([[1.] for _ in range(vect_dP_1_.shape[1])])
L = np.dot(vect_dP_1_, vect_e)
print('L =', L[0, 0])

L = 80.3573541693


In [39]:
# Дисперсия числа запросов в системе
dispV = np.dot(vect_ddP_1_, vect_e)[0] + L - L ** 2
print('V =', dispV[0, 0])

V = 303.681861894


In [40]:
# Вероятность того, что прибор 1 исправен и обслуживает запрос
temp_matr = np.dot(vect_P_1_, la.block_diag(np.eye(a*M1), np.zeros((a*(M2*L2 + M2*R + R*L1), a*(M2*L2 + M2*R + R*L1)))))
vect_e = np.array([[1.] for i in range(0, temp_matr.shape[1])])
prob1work = np.dot(temp_matr, vect_e)
print('P^{(1,0)} =', prob1work[0, 0])

P^{(1,0)} = 0.822226634202


In [41]:
# Вероятность того, что прибор-1 в неисправном состоянии и прибор-2 обслуживает запрос
temp_matr = np.dot(vect_P_1_, la.block_diag(np.zeros((a*M1, a*M1)), np.eye(a*M2*(L2 + R)), np.zeros((a*R*L1, a*R*L1))))
vect_e = np.array([[1.] for i in range(0, temp_matr.shape[1])])
prob1notwork = np.dot(temp_matr, vect_e)
print('P^{(2,0),(1,2)} =', prob1notwork[0, 0])

P^{(2,0),(1,2)} = 0.16335266263


In [42]:
# Вероятность того, что в системе есть запросы, прибор 1 в неисправном состоянии и идет переключение с этого прибора на прибор 2 (при этом оба  прибора не обслуживают заявки)
temp_matr = np.dot(vect_P_1_, la.block_diag(np.zeros((a*(M1+M2*(L2+R)), a*(M1+M2*(L2+R)))), np.eye(a*R*L1)))
vect_e = np.array([[1.] for i in range(0, temp_matr.shape[1])])
prob1notworkswitch2 = np.dot(temp_matr, vect_e)
print('P^{(2,1)} =', prob1notworkswitch2[0, 0])

P^{(2,1)} = 0.0124952081646


In [43]:
# Вероятность того, что в системе есть запросы, прибор 1 в исправном состоянии и идет переключение с  прибора 2 на прибор 1 (при этом прибор 2 продолжает обслуживать запросы)
temp_matr = np.dot(vect_P_1_, la.block_diag(np.zeros((a*M1, a*M1)), np.eye(a*M2*L2),
                                            np.zeros((a*(R*L1 + M2*R), a*(R*L1 + M2*R)))))
vect_e = np.array([[1.] for i in range(0, temp_matr.shape[1])])
prob1workswitch21 = np.dot(temp_matr, vect_e)
print('P^{(2,2)} =', prob1workswitch21[0, 0])

P^{(2,2)} = 0.0087156667819


In [44]:
# Вероятность того, что прибор 1 доступен (средняя доля времени, в течение которого прибор 1 доступен)
temp_matr1 = np.dot(vect_P_1_, la.block_diag(np.eye(a*M1), np.zeros((a*(M2*L2 + M2*R + R*L1), a*(M2*L2 + M2*R + R*L1)))))
vect_e = np.array([[1.] for i in range(0, temp_matr1.shape[1])])
temp_matr = np.dot(temp_matr1, vect_e)

temp_matr2 = np.dot(vect_p_l[0], la.block_diag(np.eye(a), np.zeros((a*(L2+R+R*L1), a*(L2+R+R*L1)))))
vect_e = np.array([[1.] for i in range(0, temp_matr2.shape[1])])
temp_matr += np.dot(temp_matr2, vect_e)
prob1avail = temp_matr
print('P_1+ =', prob1avail[0, 0])

P_1+ = 0.824044671594


In [45]:
# Вероятность того, что прибор 1 недоступен (средняя доля времени, в течение которого прибор 1 недоступен)
temp_matr1 = np.dot(vect_P_1_, la.block_diag(np.zeros((a*M1, a*M1)), np.eye(a*(M2*L2 + M2*R + R*L1))))
vect_e = np.array([[1.] for i in range(0, temp_matr1.shape[1])])
temp_matr = np.dot(temp_matr1, vect_e)

temp_matr2 = np.dot(vect_p_l[0], la.block_diag(np.zeros((a, a)), np.eye(a*(L2+R+R*L1))))
vect_e = np.array([[1.] for i in range(0, temp_matr2.shape[1])])
temp_matr += np.dot(temp_matr2, vect_e)
prob1notavail = temp_matr
print('P_2 =', prob1notavail[0, 0])

P_2 = 0.175955328406


In [46]:
temp_matr1 = np.dot(vect_P_1_, la.block_diag(np.zeros((a*M1, a*M1)), np.eye(a*M2*(L2+R)), np.zeros((a*R*L1, a*R*L1))))
vect_e = np.array([[1.] for i in range(0, temp_matr1.shape[1])])
temp_matr = np.dot(temp_matr1, vect_e)

temp_matr2 = np.dot(vect_p_l[0], la.block_diag(np.zeros((a, a)), np.eye(a*(L2+R)), np.zeros((a*R*L1, a*R*L1))))
vect_e = np.array([[1.] for i in range(0, temp_matr2.shape[1])])
temp_matr += np.dot(temp_matr2, vect_e)

prob2_avail = temp_matr
print('P_2 =', prob2_avail[0, 0])

P_2 = 0.163437413637


In [47]:
temp_matr1 = np.dot(vect_P_1_, la.block_diag(np.zeros((a*(M1+M2*(L2+R)), a*(M1+M2*(L2+R)))), np.eye(a*R*L1)))
vect_e = np.array([[1.] for i in range(0, temp_matr1.shape[1])])
temp_matr = np.dot(temp_matr1, vect_e)

temp_matr2 = np.dot(vect_p_l[0], la.block_diag(np.zeros((a*(1+L2+R), a*(1+L2+R))), np.eye(a*R*L1)))
vect_e = np.array([[1.] for i in range(0, temp_matr2.shape[1])])
temp_matr += np.dot(temp_matr2, vect_e)

prob_both_not_avail = temp_matr
print('P- =', prob_both_not_avail[0, 0])

P- = 0.012517914769


In [48]:
temp_matr1 = np.dot(vect_P_1_, la.block_diag(kron(kron(np.eye(W_), matrH1), np.eye(M1)), np.zeros((a*(M2*(L2+R)+R*L1), a*(M2*(L2+R)+R*L1)))))
vect_e = np.array([[1.] for i in range(0, temp_matr1.shape[1])])
temp_matr = np.dot(temp_matr1, vect_e)

temp_matr2 = np.dot(vect_p_l[0], la.block_diag(kron(np.eye(W_), matrH1), np.zeros((a*(L2+R+R*L1), a*(L2+R+R*L1)))))
vect_e = np.array([[1.] for i in range(0, temp_matr2.shape[1])])
temp_matr += np.dot(temp_matr2, vect_e)

switches12_num = temp_matr
print('Khi_1_2 =', switches12_num[0, 0])

Khi_1_2 = 2.59565580937


In [48]:
temp_matr1 = np.dot(vect_P_1_, la.block_diag(np.zeros((a*(M1+M2*L2), a*(M1+M2*L2))), kron(np.eye(a*M2), matrT),
                                             np.zeros((a*R*L1, a*R*L1))))
vect_e = np.array([[1.] for i in range(0, temp_matr1.shape[1])])
temp_matr = np.dot(temp_matr1, vect_e)
temp_matr2 = np.dot(vect_p_l[0], la.block_diag(np.zeros((a*(1+L2), a*(1+L2))), kron(np.eye(a), matrT),
                                               np.zeros((a*R*L1, a*R*L1))))
vect_e = np.array([[1.] for i in range(0, temp_matr2.shape[1])])
temp_matr += np.dot(temp_matr2, vect_e)
switches21_num = temp_matr
print('Khi_2_1 =', -switches21_num[0, 0])

Khi_2_1 = 1.50029920262
